# Query Incidents CSV with SQL
This notebook loads `data/Incidents.csv`, registers it as a SQL table, runs example SQL queries, and persists the table for reuse.

## Import Required Libraries
We use `pandas` for CSV loading and `duckdb` for SQL queries over DataFrames.

In [1]:
import pandas as pd
import duckdb

## Load `incidents.csv` into a DataFrame
Read the CSV, inspect the schema, and preview a few rows.

In [2]:
csv_path = "data/Incidents.csv"

df = pd.read_csv(csv_path)
df.head()


,IncidentDateTime,City,IncidentState,Country,Shape,DurationSeconds,Comments
0,2005-10-31 18:00:00.000,poughkeepsie,ny,us,light,37800.0,Several bright lights moving erratically for e...
1,2005-10-31 18:30:00.000,linwood,nj,us,light,5.0,VERY bright apparent meteor over Southern New ...
2,2005-10-31 19:00:00.000,clarksville,md,us,other,5.0,White ball shaped bright object whizzing acros...
3,2005-10-31 19:00:00.000,newark,de,us,light,45.0,Very fast&#44 brillant bluish/white light trav...
4,2005-10-31 19:00:00.000,scottsdale,az,us,triangle,600.0,Gilbert


In [3]:
df.info()

df.dtypes

<class 'pandas.DataFrame'>
RangeIndex: 6452 entries, 0 to 6451
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IncidentDateTime  6452 non-null   str    
 1   City              6452 non-null   str    
 2   IncidentState     6202 non-null   str    
 3   Country           5702 non-null   str    
 4   Shape             6310 non-null   str    
 5   DurationSeconds   6452 non-null   float64
 6   Comments          6451 non-null   str    
dtypes: float64(1), str(6)
memory usage: 353.0 KB


IncidentDateTime        str
City                    str
IncidentState           str
Country                 str
Shape                   str
DurationSeconds     float64
Comments                str
dtype: object

## Register DataFrame as an SQL Table
Register the DataFrame in DuckDB so it can be queried with SQL.

In [4]:
con = duckdb.connect()
con.register("incidents", df)

con.execute("SELECT COUNT(*) AS total_rows FROM incidents").df()

,total_rows
0,6452


## Run SQL Queries Against the Table
Use SQL to select, filter, and group data.

Note: this notebook uses DuckDB SQL (not SQL Server T-SQL). Use standard SQL functions like `COALESCE` instead of `ISNULL`.

In [5]:
con.execute("PRAGMA table_info('incidents')").df()

,cid,name,type,notnull,dflt_value,pk
0,0,IncidentDateTime,VARCHAR,False,None,False
1,1,City,VARCHAR,False,None,False
2,2,IncidentState,VARCHAR,False,None,False
3,3,Country,VARCHAR,False,None,False
4,4,Shape,VARCHAR,False,None,False
5,5,DurationSeconds,DOUBLE,False,None,False
6,6,Comments,VARCHAR,False,None,False


In [6]:

con.execute("""SELECT Shape,
       AVG(DurationSeconds) AS Average, 
       MIN(DurationSeconds) AS Minimum, 
       MAX(DurationSeconds) AS Maximum
FROM Incidents
GROUP BY Shape
-- Return records where minimum of DurationSeconds is greater than 1
having min(DurationSeconds) > 1""").df()


,Shape,Average,Minimum,Maximum
0,chevron,1100.593750,2.0,21600.0
1,cross,848.133333,2.0,7200.0
2,changing,3191.674419,2.0,172800.0
3,egg,558.956140,1.5,7200.0
4,rectangle,969.613208,4.0,28800.0
5,cylinder,795.241758,3.0,37800.0
6,teardrop,3501.685185,2.0,172800.0
7,crescent,10.000000,10.0,10.0


In [7]:
con.execute("""SELECT avg(DurationSeconds) AS Average, 
       min(DurationSeconds) AS Minimum, 
       max(DurationSeconds) AS Maximum
FROM Incidents""").df()

,Average,Minimum,Maximum
0,5592.875783,0.08,10526400.0


In [8]:
con.execute("""
SELECT IncidentState, COALESCE(IncidentState, City) AS Location
FROM incidents
-- Filter to only return missing values from IncidentState
WHERE IncidentState IS NULL""").df()

,IncidentState,Location
0,None,australia
1,None,ivanka pri dunaji (slovakia)
2,None,tunisia (in-flight; over ocean)
3,None,dehradun (uttar pradesh) (india)
4,None,erode (india)
...,...,...
245,None,sarpsborg (norway)
246,None,broken hill
247,None,taipei (taiwan)
248,None,sundra/delmas (south africa)


In [9]:
con.execute("""SELECT Country, COALESCE(Country, IncidentState, City) AS Location
FROM Incidents
WHERE Country IS NULL""").df()

,Country,Location
0,None,australia
1,None,ivanka pri dunaji (slovakia)
2,None,tunisia (in-flight; over ocean)
3,None,dehradun (uttar pradesh) (india)
4,None,erode (india)
...,...,...
745,None,tx
746,None,ca
747,None,md
748,None,ms


In [10]:
con.execute("""SELECT Country, 
       CASE WHEN Country = 'us'  THEN 'USA'
       ELSE 'International'
       END AS SourceCountry
FROM Incidents""").df().head()

,Country,SourceCountry
0,us,USA
1,us,USA
2,us,USA
3,us,USA
4,us,USA


In [11]:
con.execute("""-- Complete the syntax for cutting the duration into different cases
SELECT DurationSeconds, 
-- Start with the 2 TSQL keywords, and after the condition a TSQL word and a value
      case when (DurationSeconds <= 120) then 1
-- The pattern repeats with the same keyword and after the condition the same word and next value          
       when (DurationSeconds > 120 AND DurationSeconds <= 600) then 2
-- Use the same syntax here             
       when (DurationSeconds > 601 AND DurationSeconds <= 1200) then 3
-- Use the same syntax here               
       when (DurationSeconds > 1201 AND DurationSeconds <= 5000) then 4
-- Specify a value      
       ELSE 5 
       END AS SecondGroup   
FROM Incidents""").df().head()

,DurationSeconds,SecondGroup
0,37800.0,5
1,5.0,1
2,5.0,1
3,45.0,1
4,600.0,2


In [12]:
csv_path = "data/MixData.csv"

df = pd.read_csv(csv_path)
df.head()


,MixId,MixDesc,plant,ShipDate,DeliveryWeight,Cost,Quantity,OrderDate,WeightValue
0,100900,ABC SLURRY,1,2017-09-28T08:50:26Z,3848.800,11.2480,27.0110,2017-09-27T06:50:26Z,2.19691
1,100900,ABC SLURRY,2,2016-06-24T10:48:19Z,3848.800,10.3976,27.0110,2016-06-23T08:48:19Z,2.19691
2,105900,1/2 SACK ABC SLURRY,1,2016-06-16T15:05:09Z,3855.800,13.2444,27.0089,2016-06-16T09:05:09Z,2.19890
3,165899,6.0 SACK GROUT-NO ADMIX,2,2016-06-24T10:58:49Z,3835.905,36.0652,27.0150,2016-06-23T13:58:49Z,2.19322
4,115930,1.5 SACK APS ENCASEMENT,1,2016-06-19T10:36:08Z,3870.800,15.6100,27.0087,2016-06-18T08:36:08Z,2.20318


In [13]:
con1 = duckdb.connect()
con1.register("shipments", df)

con1.execute("SELECT COUNT(*) AS total_rows FROM shipments").df()

,total_rows
0,1143


In [14]:

con1.execute("""SELECT OrderDate, ShipDate, 
       datediff(DD, OrderDate, ShipDate) AS Duration
FROM Shipments""").df().head()

BinderException: Binder Error: Referenced column "DD" not found in FROM clause!
Candidate bindings: "MixDesc", "ShipDate", "DeliveryWeight"

LINE 2:        datediff(DD, OrderDate, ShipDate) AS Duration
                        ^

In [ ]:
con1.execute("""-- Return the DeliveryDate as 5 days after the ShipDate
SELECT OrderDate, 
       DATEADD(DD,5,ShipDate) AS DeliveryDate
FROM Shipments""").df().head()

CatalogException: Catalog Error: Scalar Function with name dateadd does not exist!
Did you mean "date_add"?

LINE 3:        DATEADD(DD,5,ShipDate) AS DeliveryDate
               ^

In [ ]:
DECLARE @counter INT 
SET @counter = 20

-- Create a loop
while @counter<30
begin
-- Loop code starting point

	SELECT @counter = @counter + 1
-- Loop finish
end

-- Check the value of the variable
SELECT @counter

SyntaxError: invalid syntax (3158411219.py, line 1)

In [ ]:
SELECT a.RecordId, a.Age, a.BloodGlucoseRandom, 
-- Select maximum glucose value (use colname from derived table)    
       b.MaxGlucose
FROM Kidney a
-- Join to derived table
join (SELECT Age, MAX(BloodGlucoseRandom) AS MaxGlucose FROM Kidney GROUP BY Age) b
-- Join on Age
on a.Age=b.Age

In [ ]:
SELECT *
FROM Kidney a
-- Create derived table: select age, max blood pressure from kidney grouped by age
JOIN (select age, max(BloodPressure) as MaxBloodPressure from kidney GROUP BY age) b
-- JOIN on BloodPressure equal to MaxBloodPressure
ON a.BloodPressure=(b.MaxBloodPressure)
-- Join on Age
AND a.age=b.age

In [ ]:
-- Specify the keyowrds to create the CTE
with  BloodGlucoseRandom (MaxGlucose) 
as (SELECT MAX(BloodGlucoseRandom) AS MaxGlucose FROM Kidney)

SELECT a.Age, b.MaxGlucose
FROM Kidney a
-- Join the CTE on blood glucose equal to max blood glucose
JOIN BloodGlucoseRandom b
on a.BloodGlucoseRandom=b.MaxGlucose

In [ ]:
-- Create the CTE
WITH BloodPressure 
AS (select max(BloodPressure) as MaxBloodPressure from Kidney)

SELECT *
FROM Kidney a
-- Join the CTE  
join BloodPressure b 
on a.BloodPressure=b.MaxBloodPressure

In [ ]:
SELECT OrderID, TerritoryName, 
       -- Total price for each partition
       sum(OrderPrice) 
       -- Create the window and partitions
       Over (partition by TerritoryName) AS TotalPrice
FROM Orders

In [ ]:
SELECT OrderID, TerritoryName, 
       -- Number of rows per partition
       count(*) 
       -- Create the window and partitions
       over (partition by TerritoryName) AS TotalOrders
FROM Orders